In [1]:
#Imports & Configuration

In [2]:
import requests

In [3]:
import pandas as pd

In [4]:
import numpy as np

In [5]:
import datetime

In [6]:
# See columns in the Dataframe

In [8]:
pd.set_option('display.max_columns', None)

In [9]:
pd.set_option('display.max_colwidth', None)

In [10]:
print("Libraries have been imported!")

Libraries have been imported!


In [11]:
# Specify the SpaceX API URL

In [12]:
spacex_url = "https://api.spacexdata.com/v4/launches/past"

In [13]:
print("Dataset URL successfully fetched")

Dataset URL successfully fetched


In [14]:
# Translate return IDs to Understandable Column Names (RocketID to Booster Name, LaunchSite_Data to Launchpad Names, Payload_Data to Payload Mass and Orbit, Landing_Outcomes to True/False)

In [15]:
# Rocket ID to Booster Name

In [53]:
def getBoosterVersion(data):
    """Fetches the Booster name from the Rocket ID"""
    for x in data:
        if x['rocket']:
            response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x['rocket'])).json()
            x['rocket'] = response['name']

In [54]:
# LaunchSite_Data to Launchpad Name, Longitude and Latitude

In [55]:
def getLaunchSite(data):
    """Fetches the LAunchpad name, Longitude and Latitude"""
    for x in data:
        if x['launchpad']:
            response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x['launchpad'])).json()
            x['launchpad'] = response['name']
            x['longitude'] = response['longitude']
            x['latitude'] = response['latitude']

In [56]:
# PayLoad_Data to Payload Mass and Orbit type

In [66]:
def getPayloadData(data):
    """Fetches the Payload Mass and Orbit type"""
    for x in data:
        if x['payloads']:
            response = requests.get("https://api.spacexdata.com/v4/payloads/"+str(x['payloads'][0])).json()
            x['payload_mass'] = response['mass_kg']
            x['orbit'] = response['orbit']

In [67]:
# Core_Data to Launch Outcome True/False

In [68]:
def getCoreData(data):
    """Fetches the Launch Outcome Data by True/False using If/Else"""
    for x in data:
        if x['cores']:
            core = x['cores'][0]
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+str(core['core'])).json()
                x['block'] = response['block']
                x['reuse_count'] = response['reuse_count']
                x['serial'] = response['serial']
            else:
                x['block'] = None
                x['reuse_count'] = None
                x['serial'] = None
            x['outcome'] = core['landing_success']
            x['flights'] = core['flight']
            x['gridfins'] = core['gridfins']
            x['reused'] = core['reused']
            x['legs'] = core['legs']
            x['landingpad'] = core['landpad']

print("Column Names Defined")

Column Names Defined


In [69]:
# Fetch and Filter Data for Specific Launch Vehicle (Falcan 9) not other launches. Vehicle lists (https://www.spacex.com/launches) / https://www.spacex.com/vehicles/falcon-9

In [70]:
# Request the data
response = requests.get(spacex_url)
data = response.json()

In [71]:
# Convert received data to DataFrame
data = pd.DataFrame(data)

In [72]:
# Filter Launches for Falcon 9 (ID: 5e9d0d95eda69973a809d1ec)
data = data[data['rocket'] == '5e9d0d95eda69973a809d1ec']

# Filter for Launches that were successful to show
data = data[data['success'].notna()]

# Reset our list to JSON format for column names
data_json = data.to_dict(orient='records')

print(f"Filtered down to {len(data_json)} Falcon 9 launches.")

Filtered down to 178 Falcon 9 launches.


In [73]:
# Process the data from MANY API calls to look up Booster,LaunchSite,Payload and Core data

In [74]:
print("Starting Data Processing... Tasks 4/4 to complete... (This may take a while...)")

getBoosterVersion(data_json)
print("... Task 1: Booster data fetched")

getLaunchSite(data_json)
print("... Task 2: Launch Sites data fetched")

getPayloadData(data_json)
print("... Task 3: Payload data fetched")

getCoreData(data_json)
print("... Task 4: Core outcomes data fetched")

print("Tasks 4/4 Fully Processed!")

Starting Data Processing... Tasks 4/4 to complete... (This may take a while...)
... Task 1: Booster data fetched
... Task 2: Launch Sites data fetched
... Task 3: Payload data fetched
... Task 4: Core outcomes data fetched
Tasks 4/4 Fully Processed!


In [75]:
# Create dictionary / filter out data / save to file for later processing

In [76]:
launch_dict = {
    'FlightNumber': list(data['flight_number']),
    'Date': list(data['date_utc']),
    'BoosterVersion': [x['rocket'] for x in data_json],
    'PayloadMass': [x['payload_mass'] for x in data_json],
    'Orbit': [x['orbit'] for x in data_json],
    'LaunchSite': [x['launchpad'] for x in data_json],
    'Outcome': [x['outcome'] for x in data_json],
    'Flights': [x['flights'] for x in data_json],
    'GridFins': [x['gridfins'] for x in data_json],
    'Reused': [x['reused'] for x in data_json],
    'Legs': [x['legs'] for x in data_json],
    'LandingPad': [x['landingpad'] for x in data_json],
    'Block': [x['block'] for x in data_json],
    'ReusedCount': [x['reuse_count'] for x in data_json],
    'Serial': [x['serial'] for x in data_json],
    'Longitude': [x['longitude'] for x in data_json],
    'Latitude': [x['latitude'] for x in data_json]
}

In [77]:
# Convert to Dataframe
df = pd.DataFrame(launch_dict)

In [78]:
# Filter out any possible mention of Falcon 1
df = df[df['BoosterVersion'] != 'Falcon 1']

In [80]:
# Reset the Index
df.reset_index(drop=True, inplace=True)
df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,6,2010-06-04T18:45:00.000Z,Falcon 9,NaN,LEO,CCSFS SLC 40,None,1,False,False,False,None,1,0,B0003,-80.577366,28.561857
1,7,2010-12-08T15:43:00.000Z,Falcon 9,NaN,LEO,CCSFS SLC 40,None,1,False,False,False,None,1,0,B0004,-80.577366,28.561857
2,8,2012-05-22T07:44:00.000Z,Falcon 9,525.0,LEO,CCSFS SLC 40,None,1,False,False,False,None,1,0,B0005,-80.577366,28.561857
3,9,2012-10-08T00:35:00.000Z,Falcon 9,400.0,ISS,CCSFS SLC 40,None,1,False,False,False,None,1,0,B0006,-80.577366,28.561857
4,10,2013-03-01T19:10:00.000Z,Falcon 9,677.0,ISS,CCSFS SLC 40,None,1,False,False,False,None,1,0,B0007,-80.577366,28.561857


In [81]:
# Save our dictionary to CSV file for future processing
df.to_csv('1_data_collection.csv', index=False)
print("Data saved to 1_data_collection.csv")

Data saved to 1_data_collection.csv
